In [ ]:
# tree importances

In [ ]:
# random forest

In [2]:
!pip install optuna

  Using cached optuna-3.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)


In [4]:
from sklearn.ensemble import RandomForestClassifier
import optuna
import mlflow
import os
import pandas as pd
from sklearn.metrics import f1_score
pd.set_option('display.max_columns', None)

In [31]:
def get_or_create_experiment(experiment_name):
    """
    Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

    This function checks if an experiment with the given name exists within MLflow.
    If it does, the function returns its ID. If not, it creates a new experiment
    with the provided name and returns its ID.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.

    Returns:
    - str: ID of the existing or newly created MLflow experiment.
    """

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        print("vim para aqui!")
        return experiment.experiment_id
    else:
        print("estou aqui!")
        return mlflow.create_experiment(experiment_name)

In [43]:
def objective(trial):
    with mlflow.start_run(nested=True, experiment_id=337532007981365636):
        if True:
            train_path = os.path.join(os.path.join("..\\data\\03_primary\\X_train_cleaned.csv"))
            target_path = os.path.join(os.path.join("..\\data\\03_primary\\y_train_cleaned.csv"))
            # Load the data
            X_train = pd.read_csv(train_path)
            y_train = pd.read_csv(target_path)




            # load test data
            X_test = pd.read_csv(os.path.join("..\\data\\03_primary\\X_test_cleaned.csv"))
            y_test = pd.read_csv(os.path.join("..\\data\\03_primary\\y_test_cleaned.csv"))

            dataset_all_together = pd.merge(X_train, y_train, left_index=True, right_index=True)
            dataset_all_together_test = pd.merge(X_test, y_test, left_index=True, right_index=True)

            # display(X_train.head(3))
            # jasdhasjkasgashjgas
            cols_to_drop_temp = ["diag_1", "diag_2", "diag_3"]

            dataset_all_together.drop(cols_to_drop_temp, axis=1, inplace=True)
            dataset_all_together_test.drop(cols_to_drop_temp, axis=1, inplace=True)
            X_train = dataset_all_together.drop('readmitted', axis=1)
            y_train = dataset_all_together['readmitted']

            X_test = dataset_all_together_test.drop('readmitted', axis=1)
            y_test = dataset_all_together_test['readmitted']


        # Create th 
    # Define the hyperparameter space
    # n_estimators = trial.suggest_int('n_estimators', 10, 40, step=5)
    # max_depth = trial.suggest_int('max_depth', 2, 32, step=5)
    # min_samples_split = trial.suggest_int('min_samples_split', 30, 80,step=10)
    # min_samples_leaf = trial.suggest_int('min_samples_leaf', 11, 31 ,step=5)
    # max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        params = {
            "n_estimators": trial.suggest_categorical("n_estimators", [10, 20, 30, 40, 50, 100, 200]),
            "max_depth": trial.suggest_categorical("max_depth", [2, 5, 10, 20, 30, 40, 50, 100]),
            "min_samples_split": trial.suggest_categorical("min_samples_split", [10, 15, 20, 30, 40, 50, 100, 500]),
            "min_samples_leaf": trial.suggest_categorical("min_samples_leaf", [11, 21, 51, 101, 501]),
            "max_features": trial.suggest_categorical("max_features", [None, 'sqrt', 'log2', 5, 10, 15, 20]),
            "n_jobs": trial.suggest_categorical("n_jobs", [None]),
            "random_state": trial.suggest_categorical("random_state", [42]),
        }


        clf = RandomForestClassifier(
            **params
        )

        # Train the model
        clf.fit(X_train, y_train)
        
        # Make predictions
        y_pred = clf.predict(X_test)
        
        # Evaluate the model
        f1 = f1_score(y_test, y_pred, average='macro')
        
        # Log parameters and metrics to MLflow
        # mlflow.log_param('n_estimators', n_estimators)
        # mlflow.log_param('max_depth', max_depth)
        # mlflow.log_param('min_samples_split', min_samples_split)
        # mlflow.log_param('min_samples_leaf', min_samples_leaf)
        # mlflow.log_param('max_features', max_features)
        mlflow.log_params(params)
        mlflow.log_metric('f1_score', f1)
        
    return f1



In [44]:


# mlflow.set_experiment('projeto_mlops')
# get the right experiment

mlflow.set_tracking_uri("http://localhost:8080")

tracking_uri = mlflow.get_tracking_uri()
print(f"Current tracking uri: {tracking_uri}")


Current tracking uri: http://localhost:8080


In [45]:
my_experiment_id = get_or_create_experiment('mlops_project')
description = "RandomForestClassifier runned in a notebook for testing. Optuna is used for hyperparameter optimization."
name = "RandomForestClassifier on notebook"

print(f"Experiment ID: {my_experiment_id}")



vim para aqui!
Experiment ID: 337532007981365636


In [46]:
# Start an MLflow run
with mlflow.start_run(experiment_id=my_experiment_id, run_name=name, nested=True) as run:
    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)

    # Log the best parameters
    mlflow.log_params(study.best_params)
    mlflow.log_metric('best_f1_score', study.best_value)

    mlflow.set_tags(
        tags={
            "project": "mlops_project",
            "description": description,
            "model_family": "RandomForestClassifier",
            "features_version":0
        }
    )


    # save the model as an artifact
    # mlflow.sklearn.log_model(study.best_trial.user_attrs['model'], artifact_path="rf_classifier_notebook")


    # Print the best parameters
    print("Best parameters: ", study.best_params)
    print("Best F1 score: ", study.best_value)



    best_rf_aaa = RandomForestClassifier(**study.best_params)
    train_path = os.path.join(os.path.join("..\\data\\03_primary\\X_train_cleaned.csv"))
    target_path = os.path.join(os.path.join("..\\data\\03_primary\\y_train_cleaned.csv"))
    # Load the data
    X_train = pd.read_csv(train_path)
    y_train = pd.read_csv(target_path)



    # load test data
    X_test = pd.read_csv(os.path.join("..\\data\\03_primary\\X_test_cleaned.csv"))
    y_test = pd.read_csv(os.path.join("..\\data\\03_primary\\y_test_cleaned.csv"))

    dataset_all_together = pd.merge(X_train, y_train, left_index=True, right_index=True)
    dataset_all_together_test = pd.merge(X_test, y_test, left_index=True, right_index=True)

    # display(X_train.head(3))
    # jasdhasjkasgashjgas
    cols_to_drop_temp = ["diag_1", "diag_2", "diag_3"]

    dataset_all_together.drop(cols_to_drop_temp, axis=1, inplace=True)
    dataset_all_together_test.drop(cols_to_drop_temp, axis=1, inplace=True)
    X_train = dataset_all_together.drop('readmitted', axis=1)
    y_train = dataset_all_together['readmitted']

    X_test = dataset_all_together_test.drop('readmitted', axis=1)
    y_test = dataset_all_together_test['readmitted']
    best_rf.fit(X_train, y_train)
    artifact_path = "rf_classifier_notebook_best_model"
    mlflow.sklearn.log_model(
        artifact_path=artifact_path, 
        sk_model=best_rf_aaa, 
        input_example=X_train.head(3), 
        registered_model_name="rf_classifier_notebook_best_model",
        metadata = {"model_data_version": 0.000000000000000000001}
        )
    model_uri_rf = mlflow.get_artifact_uri(artifact_path)
    run_id_rf = run.info.run_id
    model_path_rf = f"runs:/{run_id_rf}/model"
    # Print the model path for reference
    print("Model URI:", model_uri_rf)
    print("Run ID:", run_id_rf)
    print("Model Path:", model_path_rf)

mlflow.end_run()

[I 2024-06-16 00:33:46,325] A new study created in memory with name: no-name-a809c030-0cd3-4eb2-8bec-50d3d5e21416
[I 2024-06-16 00:34:08,958] Trial 0 finished with value: 0.4436533095756776 and parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 100, 'min_samples_leaf': 11, 'max_features': None, 'n_jobs': None, 'random_state': 42}. Best is trial 0 with value: 0.4436533095756776.
[I 2024-06-16 00:34:17,828] Trial 1 finished with value: 0.3749667800941496 and parameters: {'n_estimators': 30, 'max_depth': 50, 'min_samples_split': 500, 'min_samples_leaf': 101, 'max_features': None, 'n_jobs': None, 'random_state': 42}. Best is trial 0 with value: 0.4436533095756776.
[I 2024-06-16 00:34:20,480] Trial 2 finished with value: 0.35124907690709356 and parameters: {'n_estimators': 40, 'max_depth': 5, 'min_samples_split': 50, 'min_samples_leaf': 21, 'max_features': 15, 'n_jobs': None, 'random_state': 42}. Best is trial 0 with value: 0.4436533095756776.
[I 2024-06-16 00:34:21,228]

Best parameters:  {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 100, 'min_samples_leaf': 11, 'max_features': None, 'n_jobs': None, 'random_state': 42}
Best F1 score:  0.4436533095756776


c:\Users\joaop\Desktop\Mestrado\1º Ano\2º Semestre\MLOps\Laboratório de MLOps\mlops\Lib\site-packages\mlflow\types\utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/06/16 00:34:53 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: NotFittedError("This RandomForestClassifier instance is not fitte

PermissionError: [WinError 5] Access is denied: 'C:\\Users\\fmppo'

In [25]:
# show url to see mlflow in the browser
print(f"mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000")

mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000


In [26]:
tracking_uri = mlflow.get_tracking_uri()
print(f"Current MLflow tracking URI: {tracking_uri}")

Current MLflow tracking URI: file:///c:/Users/joaop/Desktop/Mestrado/1%C2%BA%20Ano/2%C2%BA%20Semestre/MLOps/Laborat%C3%B3rio%20de%20MLOps/projeto-mlops/notebooks/mlruns
